# Generic Pipeline tests

## Imspector-independent stuff

In [ ]:
import numpy as np
from itertools import count
from pprint import pprint
import json
from queue import PriorityQueue
from collections import defaultdict
from time import time, sleep, clock
import signal
import hashlib
import os
from itertools import zip_longest, chain, cycle
from unittest.mock import MagicMock

#from jsonpath_ng import jsonpath, parse

from spot_util import pair_finder_inner

In [ ]:
def _relative_spiral_generator(steps, start=[0,0]):
    """
    generator for regular spiral coordinates around a starting point
    with given step sizes
    """
    n = 0
    yield start.copy()
    while True:
        bookmark = [- n * steps[0] + start[0], n * steps[1] + start[0]]
        for _ in range(2*n):
            yield bookmark.copy()
            bookmark[0] += steps[0]
        for _ in range(2*n):
            yield bookmark.copy()
            bookmark[1] -= steps[1]
        for _ in range(2*n):
            yield bookmark.copy()
            bookmark[0] -= steps[0]
        for _ in range(2*n):
            yield bookmark.copy()
            bookmark[1] += steps[1]
        n += 1


        
class LegacySpotPairFinder():
    """
    wrapper for the 'old' spot pair detector
    get_locations will return a list of coordinate lists
    of scan coordinates (stage coordinates are ignored)
    """
    def __init__(self, dataSource, sigma, thresholds, medianThresholds=[3,3], medianRadius=5):
        self.dataSource = dataSource
        self.sigma = sigma
        self.thresholds = thresholds
        self.medianThresholds = medianThresholds
        self.medianRadius = medianRadius
        self.plotDetections = False
        
    def withPlotDetections(plotDetections):
        self.plotDetections = plotDetections
        return self
    
    def get_locations(self):
        data = self.dataSource.get_data()
        if (data.numConfigurations < 1) or (data.numImages(0) < 2):
            raise ValueError('too few images for LegacySpotPairFinder. The RichData provided needs to have two images in the first configuration.')
        stack1 = data.data[0][0][0,:,:,:]
        stack2 = data.data[0][1][0,:,:,:]
        
        setts = data.measurementSettings[0]
        
        pairsRaw = pair_finder_inner(stack1, stack2, self.sigma, self.thresholds, True, False, self.medianThresholds, self.medianRadius)
        
        offsOld = np.array([filter_dict(
            setts, 'ExpControl/scan/range/{}/off'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)
        
        lensOld = np.array([filter_dict(
            setts, 'ExpControl/scan/range/{}/len'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)
        
        pszOld = np.array([filter_dict(
            setts, 'ExpControl/scan/range/{}/psz'.format(c), False) for c in ['x', 'y', 'z']], dtype=float)
        
        # TODO: do plotting
        if self.plotDetections:
            pass
        
        res = []
        for pair in pairs:
            pairT = np.array(pair, dtype=float)
            res.append(list(offsOld - (lensOld /.2) + pairT * pszOld))
        return res
            
        
class AcquisitionPriorityQueue(PriorityQueue):
    """
    slightly modified PriorityQueue to be able to enqueue non-orderable data
    """
    def __init__(self):
        PriorityQueue.__init__(self)
        self.ctr = count()
    def put(self, item, prio):
        PriorityQueue.put(self, (prio, next(self.ctr), item))
    def get(self, *args, **kwargs):
        lvl, _, item = PriorityQueue.get(self, *args, **kwargs)
        return (lvl, item)
        
class _pipeline_level:
    """
    named level in an acquisition pipeline
    should not be used outside of a PipelineLevels object
    """
    def __init__(self, parent, name):
        self.parent = parent
        self.name = name
    
    def __eq__(self, other):
        if type(self) != type(other):
            return False
        return self.name == other.name
    
    def __le__(self, other):
        return self.parent.reversedLevels.index(self) <= self.parent.reversedLevels.index(other)
    
    def __lt__(self, other):
        return self.parent.reversedLevels.index(self) < self.parent.reversedLevels.index(other)
    
    def __str__(self):
        return self.name
    
    def __hash__(self):
        return str.__hash__(self.name)
    
    def __repr__(self):
        return self.name


class DelayedKeyboardInterrupt():
    """
    context manager to allow finishing of one acquisition loop
    before quitting queue due to KeyboardInterrupt
    
    modified from https://stackoverflow.com/a/21919644
    """
    
    def __init__(self, pipeline):
        self.pipeline = pipeline
    
    def __enter__(self):
        self.old_handler = signal.getsignal(signal.SIGINT)
        signal.signal(signal.SIGINT, self.handler)

    def handler(self, sig, frame):
        self.pipeline.interrupted = True

    def __exit__(self, type, value, traceback):
        signal.signal(signal.SIGINT, self.old_handler)
    
class PipelineLevels:
    """
    ordered collection of _pipeline_level
    """
    levels = []
    def __init__(self, *args):
        for arg in args:
            lvl = _pipeline_level(self, arg)
            self.levels.append(lvl)
            setattr(self, arg, lvl)
    @property
    def reversedLevels(self):
        return list(reversed(self.levels))

class TimedStoppingCriterion():
    """
    stopping criterion to stop after a set amount of time
    """
    def __init__(self, maxtime):
        self.maxtime = maxtime
    def check(self, pipeline):
        return time() > ( pipeline.startingTime + self.maxtime )
    def desc(self, pipeline):
        return 'STOPPING PIPELINE {}: maximum time exceeded'.format(pipeline.name)
        
class InterruptedStoppingCriterion():
    """
    stopping criterion to check wether SIGINT was received and stop then
    will also reset the signal status in parent AcquisitionPipeline
    """
    def check(self, pipeline):
        return pipeline.interrupted
    def resetInterrupt(self, pipeline):
        pipeline.interrupted = False
    def desc(self, pipeline):
        return 'STOPPING PIPELINE {}: interrupted by user'.format(pipeline.name)

class AcquisitionTaskGenerator():
    def __init__(self, level, *updateGens):
        
        self.level = level
        self.updateGens = updateGens
        self.delay = 0
    
    def withDelay(self, delay):
        """
        a delay that will be added to every generated task
        (e.g. to wait for the stage to move)
        """
        self.delay = delay
        return self
    
    def __call__(self, pipeline):
        # broadcast meausurement updates ((u1, u2), (u3) -> ((u1, u3), (u2, u3)))
        
        updates = [updateGenI() for updateGenI in self.updateGens]
        
        maxMeasurements = max((len(updateI) for updateI in updates))
        cyclesMeas = [cycle(updateI) for updateI in updates]
        
        for _ in range(maxMeasurements):
            
            # broadcast configurations within measurements
            configs = [next(meas) for meas in cyclesMeas]
            maxConfigs = max((len(confI) for confI in configs))
            cyclesConfig = [cycle(confI) for confI in configs]
            
            finalConfs = []
            
            for _ in range(maxConfigs):
                
                finalConfs.append([next(upd) for upd in cyclesConfig])
            
            #print(json.dumps(finalConfs, indent=2))
            pipeline.queue.put(AcquisitionTask(self.level).withUpdates(finalConfs).withDelay(self.delay), self.level)
                
class AcquisitionTask():
    """
    a dummy acquisition task, that will repeat itself every second
    """
    
    def __init__(self, pipelineLevel):
        self.pipelineLevel = pipelineLevel
        self.measurementUpdates = []
        self.settingsUpdates = []
        self.delay = 0
        
    def withUpdates(self, updates):
        for u in updates:
            measUpdates = [m for m,s in u]
            settingsUpdates = [s for m,s in u]
            self.measurementUpdates.append(measUpdates)
            self.settingsUpdates.append(settingsUpdates)
        return self
    
    def withDelay(self, delay=0):
        self.delay = delay
        return self
        
    @property
    def numAcquisitions(self):
        return len(self.measurementUpdates)
    
    def getUpdates(self, n):
        return self.measurementUpdates[n], self.settingsUpdates[n]
    
    def getAllUpdates(self):
        return [self.getUpdates(n) for n in range(self.numAcquisitions)]
    
    # TODO: move, this should become part of an analysis callback
    def __call__(self, pipeline, *args, **kwargs):
        print('pipeline {}: do dummy acquisition on level {}'.format(pipeline.name, self.pipelineLevel))
        #sleep(1)
        pipeline.queue.put(AcquisitionTask(self.pipelineLevel).withDelay(self.delay), self.pipelineLevel)


class RichData:
    """
    wrapper for data and settings of a measurement
    holds hardware ('global') settings and per measurement settings
    for each parameter set and a list of associated data
    """
    def __init__(self):
        self.globalSettings = []
        self.measurementSettings = []
        self.data = []
    
    # TODO: remove defaults?
    def append(self, globalSettings=None, measurementSettings=None, data=None):
        self.globalSettings.append(globalSettings)
        self.measurementSettings.append(measurementSettings)
        self.data.append(data)
    
    @property
    def numConfigurations(self):
        return len(self.data)
    
    def numImages(self, n):
        if n < self.numConfigurations:
            return len(self.data[n])
        else:
            return 0
    

class DefaultNameHandler():
    """
    file name handler
    """
    
    def __init__(self, path, levels, prefix=None, ending = '.msr'):
        self.path = path
        self.levels = levels
        self.ending = ending
        if prefix is None:
            hash_object = hashlib.md5(bytes(str(time()), "utf-8"))
            hex_dig = hash_object.hexdigest()
            self.prefix = str(hex_dig)
        else:
            self.prefix = prefix
            
        if not os.path.exists(path):
            os.makedirs(path)
            
    def _mkdir_if_necessary(self):
        pass
            
    def get_filename(self, idxes):
        insert = chain.from_iterable(zip([l.name for l in self.levels.levels[0:len(idxes)]], idxes))
        insert = list(insert)
        return ((self.prefix + '_{}_{}' * len(idxes)).format(*insert) + self.ending)
    
    def get_path(self, idxes):
        return os.path.join(self.path, self.get_filename(idxes))      
        
            
class MockImspectorConnection():
    
    def __init__(self):
        self.getCurrentData = MagicMock(return_value = RichData())
        self.makeMeasurementFromTask = MagicMock(return_value = None)
        self.makeConfigurationFromTask = MagicMock(return_value = None)
        self.runCurrentMeasurement = MagicMock(return_value = None)
        self.saveCurrentMeasurement = MagicMock(return_value = None)
        self.closeCurrentMeasurement = MagicMock(return_value = None)
            

            
class ImspectorConnection():
    
    def __init__(self, im):
        self.im = im
        
    def getCurrentData(self):
        globalParams = self.im.parameters('')
        measParameters = self.im.active_measurement().parameters('')
        data = []
        for name in im.active_measurement().stack_names():
            data.append(np.copy(self.im.active_measurement().stack(name).data()))
        return globalParams, measParameters, data
    
    def makeMeasurementFromTask(self, task):
        ms = self.im.create_measurement()
        measUpdates, confUpdates = task
        measUpdates = update_dicts(*measUpdates)
        confUpdates = update_dicts(*confUpdates)
        
        # we do the update twice to also set grayed-out values
        ms.set_parameters('', measUpdates)
        ms.set_parameters('', measUpdates)
        self.im.set_parameters('', confUpdates)
        self.im.set_parameters('', confUpdates)
        
    def makeConfigurationFromTask(self, task):
        ms = self.im.active_measurement()
        ac = ms.active_configuration()
        ac = ms.clone(ac)
        ms.activate(ac)
        
        measUpdates, confUpdates = task
        measUpdates = update_dicts(*measUpdates)
        confUpdates = update_dicts(*confUpdates)
        
        # we do the update twice to also set grayed-out values
        ms.set_parameters('', measUpdates)
        ms.set_parameters('', measUpdates)
        self.im.set_parameters('', confUpdates)
        self.im.set_parameters('', confUpdates)
        
    def runCurrentMeasurement(self):
        ms = self.im.active_measurement()
        #ms.activate(ms.configuration(ms.number_of_configurations()-1))
        self.im.run(ms)
        
    def saveCurrentMeasurement(self, path):
        ms = self.im.active_measurement()
        ms.save_as(path)
        
    def closeCurrentMeasurement(self):
        ms = self.im.active_measurement()
        self.im.close(ms)

class NewestDataSelector():
    """
    
    """
    def __init__(self, pipeline, level):
        self.pipeline = pipeline
        self.lvl = level
    
    def get_data(self):
        # create index of measurement (indices of all levels until lvl)
        latestMeasurementIdx = tuple([self.pipeline.counters[l] for l in self.pipeline.pipelineLevels.levels[
                            0:self.pipeline.pipelineLevels.levels.index(self.lvl)+1]])
        return self.pipeline.data.get(latestMeasurementIdx, None)


class NewestSettingsSelector():
    def __init__(self, pipeline, level):
        self.level = level
        self.pipeline = pipeline
        
    def __call__(self):
        pipeline = self.pipeline
        latestMeasurementIdx = tuple([pipeline.counters[l] for l in pipeline.pipelineLevels.levels[
                            0:pipeline.pipelineLevels.levels.index(self.level)+1]])
        data = pipeline.data.get(latestMeasurementIdx, None)
        return [list(zip(data.measurementSettings, data.globalSettings))]
        
    
class AcquisitionPipeline():
    
    """
    the main class
    """
    
    def __init__(self, name):
        """
        construct with name
        """
        self.name = name
        
        self.pipelineLevels = None
    
        # we habe an InterruptedStoppingCriterion by default
        self.stoppingConditions = [InterruptedStoppingCriterion()]
        self.queue = AcquisitionPriorityQueue()
        self.startingTime = None
        self.counters = defaultdict(int)
        self.data = defaultdict(RichData)
        self.callbacks = defaultdict(list)
        
        # hold the Imspector connection
        self.im = MockImspectorConnection()
        
        self.logger = None
        self.nameHandler = None
    
        # the DelayedKeyboardInterrupt will indicate a received SIGINT here
        self.interrupted = False
    
    def run(self):
        """
        run the pipeline
        """
        
        # we use this context manager to handle interrupts so we can finish
        # to acquisition we are in before stopping
        with DelayedKeyboardInterrupt(self):            

            # record starting time, so we can check wether a StoppingCondition is met
            self.startingTime = time()

            lvl = None
            
            while not self.queue.empty():

                # get next task and its level
                oldlvl = lvl
                lvl, acquisition_task = self.queue.get()
                
                if oldlvl is None:
                    self.counters[lvl] = -1
                
                # reset or increment indices
                if (oldlvl != lvl):
                    for l in self.pipelineLevels.levels:
                        if l < lvl:
                            self.counters[l] = -1
                
                self.counters[lvl] += 1
                                
                # create index of measurement (indices of all levels until lvl)
                currentMeasurementIdx = tuple([self.counters[l] for l in self.pipelineLevels.levels[
                            0:self.pipelineLevels.levels.index(lvl)+1]])

                # go through updates sequentially (we might have multiple configurations per measurement)
                for updatesI in range(acquisition_task.numAcquisitions):
                    
                    # update imspector
                    if updatesI == 0:
                        self.im.makeMeasurementFromTask(acquisition_task.getUpdates(updatesI))
                    else:
                        self.im.makeConfigurationFromTask(acquisition_task.getUpdates(updatesI))
                    
                    # we might want to sleep
                    sleep(acquisition_task.delay)
                    
                    # run in imspector
                    self.im.runCurrentMeasurement()
                                
                    # add data copy (of most recent configuration) to internal storage
                    self.data[currentMeasurementIdx].append(*self.im.getCurrentData())
                
                # save and close in imspector
                path = None
                if self.nameHandler != None:
                    path = self.nameHandler.get_path(currentMeasurementIdx)
                print(path)
                
                # TODO: closing without saving might trigger UI dialog in Imspector
                if not (path is None):
                    self.im.saveCurrentMeasurement(path)
                self.im.closeCurrentMeasurement()

                # do the callbacks (this should do analysis and re-fill the queue)
                callbacks_ = self.callbacks.get(lvl, None)
                if not (callbacks_ is None):
                    for callback_ in callbacks_:                    
                        callback_(self)

                # go through stopping conditions
                for sc in self.stoppingConditions:
                    if sc.check(self) == True:
                        # reset interrupt flag if necessary
                        if isinstance(sc, InterruptedStoppingCriterion):
                            sc.resetInterrupt(self)
                        print(sc.desc(self))
                        break
                # we went through all the loop iterations (no break)
                else:
                    continue
                break
            
            print('PIPELINE {} FINISHED'.format(self.name))


              
    def withPipelineLevels(self, lvls):
        """
        set pipeline levels, can be chained
        """
        self.pipelineLevels = lvls
        return self
    
    def withNameHandler(self, nh):
        self.nameHandler = nh
        return self
    
    def withImspectorConnection(self, im):
        self.im = im
        return self
    
    def withCallbackAtLevel(self, callback, lvl):
        """
        set the callback for a level, can be chained
        """
        if not (lvl in self.pipelineLevels.levels):
            raise ValueError('{} is not a registered pipeline level'.format(lvl))
        self.callbacks[lvl].append(callback)
        return self
    
    def _withStoppingConditions(self, conds):
        """
        reset the StoppingConditions, can be chained
        """
        self.stoppingConditions.clear()
        for condI in conds:
            self.stoppingConditions.append(condI)
        return self
    
    def withAddedStoppingCondition(self, cond):
        """
        add a StoppingCondition, can be chained
        """
        self.stoppingConditions.append(cond)
        return self
    
    def withInitialTask(self, task, lvl):
        """
        initialize the queue with the given task at the given level, can be chained
        """
        self.queue = AcquisitionPriorityQueue()
        self.queue.put(task, lvl)
        return self
    
class JSONFileConfigLoader():
    """
    load settings from JSON dump
    """
    def __init__(self, measurementConfigFileNames, settingsConfigFileNames=None, asMeasurements=True):
        self.measConfigs = []
        self.asMeasurements = asMeasurements
        for mFile in measurementConfigFileNames:
            with open(mFile, 'r') as fd:
                d = json.load(fd)
                # remove, otherwise Imspector complains that those parameters do not exist (yet?)
                d = remove_filter_from_dict(d, '/Measurement/LoopMeasurement')
                d = remove_filter_from_dict(d, '/Measurement/ResumeIdx')
                # remove, otherwise we will always use a set propset
                d = remove_filter_from_dict(d, '/Measurement/propset_id')
                self.measConfigs.append(d)
        
        self.settingsConfigs = []
        if settingsConfigFileNames is None:
            for _ in range(len(self.measConfigs)):
                self.settingsConfigs.append(dict())
        else:
            if len(settingsConfigFileNames) != len(self.measConfigs):
                raise ValueError('length of settings and measurement configs dont match')
            for sFile in settingsConfigFileNames:
                with open(sFile, 'r') as fd:
                    self.settingsConfigs.append(json.load(fd))
    
    def __call__(self):
        res = []
        if self.asMeasurements:
            for i in range(len(self.measConfigs)):
                res.append([(self.measConfigs[i], self.settingsConfigs[i])])
        else:
            resInner = []
            for i in range(len(self.measConfigs)):
                resInner.append([(self.measConfigs[i], self.settingsConfigs[i])])
            res.append(resInner)
        return res
                
class DefaultScanOffsetsSettingsGenerator():
    
    _paths = ['ExpControl/scan/range/x/off',
              'ExpControl/scan/range/y/off',
              'ExpControl/scan/range/z/off'
             ]
    
    def __init__(self, locationGenerator, asMeasurements=True, fun=None):
        self.locationGenerator = locationGenerator
        self.asMeasurements = asMeasurements
        if fun is None:
            self.fun = locationGenerator.get_locations
        else:
            self.fun = fun
    
    def __call__(self):
        locs = self.fun()
        
        res = []
        for loc in locs:
            resD = {}
            path = cycle(self._paths)
            for l in loc:
                resD = update_dicts(resD, gen_json(l, next(path)))
            res.append([(resD, {})])
        if self.asMeasurements:
            return res
        else:
            return [reduce(add, res)]
        
        

class DefaultStageOffsetsSettingsGenerator(DefaultScanOffsetsSettingsGenerator):
    _paths = ['ExpControl/scan/range/offsets/coarse/x/g_off',
             'ExpControl/scan/range/offsets/coarse/y/g_off',
             'ExpControl/scan/range/offsets/coarse/z/g_off']




class DefaultLocationRemover():
    """
    this wrapper can be used to remove location-related updates from the output
    of a settings generator.
    if will remove the corresponding settings from every measurement dict
    and leave the rest as-is.
    """
    
    _filtersToRemove = ['ExpControl/scan/range/offsets',
                       'ExpControl/scan/range/x/off',
                       'ExpControl/scan/range/x/g_off',
                       'ExpControl/scan/range/y/off',
                       'ExpControl/scan/range/y/g_off',
                       'ExpControl/scan/range/z/off',
                       'ExpControl/scan/range/z/g_off',
                       'OlympusIX/stage',
                       'OlympusIX/scanrange']
    
    def __init__(self, coordinateProvider):
        self.coordinateProvider = coordinateProvider
    
    def __call__(self):
        res = []
        for l in self.coordinateProvider():
            lModified = []
            for meas, settings in l:
                measI = deepcopy(meas)
                for f in DefaultLocationRemover._filtersToRemove:
                    measI = remove_filter_from_dict(measI, f)
                    if measI is None:
                        measI = {}
                lModified.append((measI, settings))
                
            res.append(lModified)
        return res
    
def get_current_stage_coords(im = None):
    
    if im is None:
        im = Imspector()
        
    im.create_measurement()
    ms = im.active_measurement()
    
    coords = [ms.parameters('ExpControl/scan/range/offsets/coarse/'+ c + '/g_off') for c in 'xyz']
    
    im.close(ms)
    
    return coords

def dump_JSON(d, path):
    """
    helper function to dump a dict to a file given by path as JSON
    """
    with open(path, 'w') as fd:
        json.dump(d, fd, indent=2)
        
class SpiralOffsetGenerator():
    def __init__(self):
        self.fov = [5e-5, 5e-5]
        self.start = [0, 0]
        self.gen = _relative_spiral_generator(self.fov, self.start)
    def withFOV(self, fov):
        self.fov = fov
        self.gen = _relative_spiral_generator(self.fov, self.start)
        return self
    def withStart(self, start):
        self.start = start
        self.gen = _relative_spiral_generator(self.fov, self.start)
        return self
    def get_locations(self):
        return [next(self.gen)]
    
import collections
from copy import deepcopy

def update_dicts(*dicts):
    if len(dicts) == 0:
        return {}
    
    first = dicts[0]
    if len(dicts) < 2:
        return deepcopy(first)
    else:
        second = dicts[1]
        return update_dicts(update_dict_pair(first, second), *dicts[2:])

def update_dict_pair(d1, d2):
    res = deepcopy(d1)
    for k, v in d2.items():
        if (isinstance(v, collections.Mapping)):
            res[k] = update_dict_pair(res.get(k) if isinstance(res.get(k, None), collections.Mapping) else {}, v)
        else:
            res[k] = v
    return res

def remove_filter_from_dict(d, flt, sep='/'):
    
    flt_strp = flt.strip(sep)
    flts = flt_strp.split(sep)    
    fst_flt = flts[0]
    
    if fst_flt == '':
        return None
    
    if len(flts) == 1:
        if (isinstance(d, collections.Sequence) and fst_flt.isdigit()):
            try:
                cpy = deepcopy(d)
                cpy.pop(int(fst_flt))
                return cpy if len(cpy) > 0 else None
            except IndexError:
                return deepcopy(d)
        elif (isinstance(d, collections.Mapping)):
            try:
                cpy = deepcopy(d)
                del cpy[fst_flt]
                return cpy if len(cpy) > 0 else None                
            except KeyError:
                return deepcopy(d)
        else:
            return deepcopy(d)
    
    if (isinstance(d, collections.Sequence) and fst_flt.isdigit()):
        try:
            res = remove_filter_from_dict(d[int(fst_flt)], sep.join(flts[1:]))
            cpy = deepcopy(d)
            if res is None:
                del cpy[int(fst_flt)]
            else:
                cpy[int(fst_flt)] = res
            return cpy if len(cpy) > 0 else None
        except IndexError:
            return deepcopy(d)
        
    elif (isinstance(d, collections.Mapping)):
        try:
            res = remove_filter_from_dict(d[fst_flt], sep.join(flts[1:]))
            cpy = deepcopy(d)
            if res is None:
                del cpy[fst_flt]
            else:
                cpy[fst_flt] = res
            return cpy if len(cpy) > 0 else None
        except KeyError:
            return deepcopy(d)
    else:
        return None    
    
    
        
def filter_dict(d, flt, keepStructure=True, sep='/'):
    
    flt_strp = flt.strip(sep)
    flts = flt_strp.split(sep)
    
    fst_flt = flts[0]
    
    if fst_flt == '':
        return d
    
    if len(flts) == 1:
        if (isinstance(d, collections.Sequence) and fst_flt.isdigit()):
            try:
                return [d[int(fst_flt)]] if keepStructure else d[int(fst_flt)]
            except IndexError:
                return None
        elif (isinstance(d, collections.Mapping)):
            try:
                return {fst_flt: d[fst_flt]} if keepStructure else d[fst_flt]
            except KeyError:
                return None
        else:
            return None
        
    if (isinstance(d, collections.Sequence) and fst_flt.isdigit()):
        try:
            res = filter_dict(d[int(fst_flt)], sep.join(flts[1:]), keepStructure)
            if res is None:
                return None
            return [res] if keepStructure else res
        except IndexError:
            return None
    elif (isinstance(d, collections.Mapping)):
        try:
            res = filter_dict(d[fst_flt], sep.join(flts[1:]), keepStructure)
            if res is None:
                return None
            return {fst_flt: res} if keepStructure else res
        except KeyError:
            return None
    else:
        return None    
    
def gen_json(data, path, sep='/'):
    
    path_strp = path.strip(sep)
    paths = path_strp.split(sep)    
    fst_path = paths[0]
    
    if fst_path == '':
        return data
    else:
        return {fst_path: gen_json(data, sep.join(paths[1:]))}

In [ ]:
DefaultLocationRemover(JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/atest.json', 'C:/Users/RESOLFT/Desktop/atest.json']))()

In [ ]:
m = MagicMock()
AcquisitionTaskGenerator("a", lambda: [[(1,2)], [(2,3)]], lambda: [[(5,6)]])(m)
c = m.method_calls[0]

print(c[1][0].getUpdates(0))
print(c[1][0].numAcquisitions)

len(m.method_calls)

from operator import add
from functools import reduce


    
#pprint(DefaultLocationRemover(lambda: [[(ms.parameters(''), {})]])())


g = _relative_spiral_generator([2,2])
a = DefaultStageOffsetsSettingsGenerator(None, fun=lambda: [next(g)])

atg = AcquisitionTaskGenerator(1, a)
m = MagicMock()

atg(m)
atg(m)

DefaultScanOffsetsSettingsGenerator(LegacySpotPairFinder(NewestDataSelector(m, "overview"), 3, [.01, .01]))



In [ ]:
from specpy import *
im = Imspector()
ms = im.active_measurement()
dump_JSON(ms.parameters(''), 'C:/Users/RESOLFT/Desktop/config_json/goldbeads_overview.json')

In [ ]:
pll = PipelineLevels('overview', 'detail')





atg = (AcquisitionTaskGenerator(pll.overview, 
                               DefaultLocationRemover(JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/goldbeads_overview.json'])),
                               DefaultStageOffsetsSettingsGenerator(SpiralOffsetGenerator().withStart(get_current_stage_coords(im))))
       .withDelay(.5))



pl = (AcquisitionPipeline('1')
        .withImspectorConnection(ImspectorConnection(im))
        .withPipelineLevels(pll)
        .withNameHandler(DefaultNameHandler('C:/Users//RESOLFT/Desktop/TEST_GEN/', pll))
        .withAddedStoppingCondition(TimedStoppingCriterion(25))
        .withCallbackAtLevel(atg, pll.overview))

atg2 = AcquisitionTaskGenerator(pll.detail,
                               NewestSettingsSelector(pl, pll.overview))

pl.withCallbackAtLevel(atg2, pll.overview)

atg(pl)

pl.run()


In [ ]:
               
            
            
pll = PipelineLevels('overview', 'detail', 'sted')
tsk = AcquisitionTask(pll.overview).withDelay(.0)

pl = (AcquisitionPipeline('1')
        .withImspectorConnection(ImspectorConnection(im))
        .withPipelineLevels(pll)
        .withNameHandler(DefaultNameHandler('C:/Users//RESOLFT/Desktop/TEST_GEN/', pll))
        .withAddedStoppingCondition(TimedStoppingCriterion(15))
        .withInitialTask(tsk, pll.overview)
        .withCallbackAtLevel(AcquisitionTask(pll.overview).withDelay(.3), pll.overview)
        .withCallbackAtLevel(lambda x: print(LegacySpotPairFinder(2, [.003, .003], [1,1]).get_locations(NewestDataSelector(pl, pll.overview).get_data())), pll.overview))
pl.run()


'''
pl = (AcquisitionPipeline('2')
        .withPipelineLevels(pll)
        .withNameHandler(DefaultNameHandler('aaa', pll))
        .withAddedStoppingCondition(TimedStoppingCriterion(5))
        .withInitialTask(tsk, pll.overview))
pl.run()
'''  


#print(pl.data[(0,)].measurementSettings[0])
print(pl.data[(0,)].numImages(0))
pl.data

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

ds = NewestDataSelector(pl, pll.overview)
ds.get_data().data[0]

plt.imshow(np.apply_along_axis(np.mean, 0, ds.get_data().data[0][0][0,:,:,:]), cmap='inferno')

LegacySpotPairFinder(2, [.001, .001], [1,1]).get_locations(ds.get_data())

In [ ]:
g = _relative_spiral_generator([2,2])
[next(g) for _ in range(12)]
tuple([1])
callable(lambda: np.arange(1,2))

# Imspector stuff from here on

In [ ]:
import specpy as sp
im = sp.Imspector()
im.version()

im.active_measurement().parameters('')

In [ ]:
ms = im.create_measurement()
im.activate(ms)

#im.run(ms)
ms.clone(ms.active_configuration())
#im.run(ms)
import json



In [ ]:

    
d1 = {'a' : {'B' : 2, 'b' : [1,2,3]}}
d2 = {'a' : {'B' : 2}}
d3 = {'a' : 3}

d = update_dicts(d1, d2)
print(d)

update_dicts(filter_dict(d1, 'a/'), d2)

print(remove_filter_from_dict(d1, '/a/B'))
filter_dict(d1, 'a/b', False)

In [ ]:
ms = im.active_measurement()
#pprint(ms.parameters(''))
upd = update_dicts(filter_dict(dict(ms.parameters('')), '', keepStructure=False))

pprint(upd)

gen = _relative_spiral_generator([1e-5, 1e-5])

for _ in range(10):
    xy = next(gen)
    upd = update_dicts(gen_json(xy[0], 'ExpControl/scan/range/offsets/coarse/x/g_off'),
                       gen_json(xy[1], 'ExpControl/scan/range/offsets/coarse/y/g_off'))

    pprint(upd)

    ms.set_parameters('', upd)
    sleep(2)

In [ ]:
#im.active_measurement().clone(im.active_measurement().active_configuration())
im.run(im.active_measurement())

In [ ]:
it = count()


ms = im.create_measurement()
ms.set_parameters('', params)

im.connect_begin(lambda : next(it), 1)
im.run(ms)

params = ms.parameters('')
js = json.dumps(ms.parameters(''), indent=2)

im.close(ms)
print(next(it))

print(js)
justoix = dict([(k,v) for k,v in params.items() if (k == 'OlympusIX')])

#pprint(im.parameters(''))